## Hacer cambios en Modelos

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(default="No description provided")  # Valor por defecto
    short_description = models.TextField(null=True)
    supplier = models.TextField(default="Unknown supplier")            # Valor por defecto
    color = models.CharField(max_length=30, default="Unknown color")   # Valor por defecto
    dimensions = models.CharField(max_length=100, default="Unknown dimensions") 

On container:
-python manage.py makemigrations
-python manage.py makemigrations


## Borrar un campo de un modelo

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(default="No description provided")  # Valor por defecto
    short_description = models.TextField(null=True)
    supplier = models.TextField(default="Unknown supplier")            # Valor por defecto
    color = models.CharField(max_length=30, default="Unknown color")   # Valor por defecto
    dimensions = models.CharField(max_length=100, default="Unknown dimensions") 

In [ ]:
On container:
    python manage.py makemigrations
  python manage.py migrate


In [ ]:
python manage.py squashmigrations <APP_LABEL> <MIGRATION_NUMBER>
python manage.py squashmigrations ecommerce 0004
python manage.py migrate

## Guardando data usando shell de Django

In [ ]:
python manage.py shell
>from ecommerce.models import ProductModel
>ProductModel.objects.create(title = "Producto 1", price = 199.99)

#### Queryset: lista de todos los objetos almacenados para cierto modelo

In [ ]:
queryset = ProductModel.objects.all()
qs = ProductModel.objects.all()

In [ ]:
queryset.filter(title__icontains="producto")

In [ ]:
my_product = ProductModel.objects.get(id = 17)

In [ ]:
>>> my_product = ProductModel.objects.get(id = 16)
>>> my_product
<ProductModel: ProductModel object (16)>
>>> my_product.title
'Producto 2'
>>> my_product.description
'Es una prueba'
>>> my_product.price
199.99
>>> my_product.price = 249.99
>>> my_product.price
249.99
>>> my_product.save()
>>> exit()


## Validacion de campos en los modelos

### ecommerce/validators

In [ ]:
from django.core.exceptions import ValidationError

BLOCKED_WORDS = [
    "barato",
    "malo"
]

def validate_blocked_words(value):
    init_string = f"{value}".lower()
    unique_words = set(init_string.split())
    blocked_words = set(BLOCKED_WORDS)
    invalid_words = (unique_words & blocked_words)
    has_error = len(invalid_words) > 0
    if has_error:
        errors = []
        for invalid_word in invalid_words:
            msg = "{} es una palabra no permitida".format(invalid_word)
            errors.append(msg)
        raise ValidationError(errors)
    return value


## ecommerce/models.py

In [ ]:
from django.db import models
from .validators import validate_blocked_words

class ProductModel(models.Model):
    title = models.CharField(max_length=255)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    description = models.TextField(null=True, blank=True)
    seller = models.CharField(max_length=255, null=True, blank=True)
    color = models.CharField(max_length=50, null=True, blank=True)
    product_dimensions = models.CharField(max_length=100, null=True, blank=True)

    def __str__(self):
        return self.title
    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        validate_blocked_words(self.description)
        super().save(*args, **kwargs)
    


## como agregar opciones a los campos de nuestros Modelos

In [ ]:
from django.db import models
from .validators import validate_blocked_words

# [(VALOR_EN_DB, VALOR_PARA_USUARIO)]
PUBLISH_STATE_CHOICES = [
    ("BR", "BORRADOR"),
    ("PU", "PUBLICADO"),
    ("PR", "PRIVADO"),
]

class ProductModel(models.Model):
    state = models.CharField(
        max_length=2,
        choices=PUBLISH_STATE_CHOICES,
        default="BR"
    )
    title = models.CharField(max_length=255)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    description = models.TextField(null=True, blank=True)
    seller = models.CharField(max_length=255, null=True, blank=True)
    color = models.CharField(max_length=50, null=True, blank=True)
    product_dimensions = models.CharField(max_length=100, null=True, blank=True)

    def __str__(self):
        return self.title
    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        validate_blocked_words(self.description)
        super().save(*args, **kwargs)
    
    def is_published(self):
        return self.state == "PU"

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Como agregar opciones avanzadas a los campos de nuestros Modelos

In [ ]:
from django.db import models
from .validators import validate_blocked_words


class ProductModel(models.Model):
    class ProductStateOptions(models.TextChoices):
        PUBLISHED = "PU", "PUBLICADO"
        DRAFT = "BR", "BORRADOR"
        PRIVATE = "PR", "PRIVADO"

    state = models.CharField(
        max_length=2, 
        choices=ProductStateOptions.choices, 
        default=ProductStateOptions.DRAFT
    )
    title = models.CharField(max_length=255)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    description = models.TextField(null=True, blank=True)
    seller = models.CharField(max_length=255, null=True, blank=True)
    color = models.CharField(max_length=50, null=True, blank=True)
    product_dimensions = models.CharField(max_length=100, null=True, blank=True)

    def __str__(self):
        return self.title

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        validate_blocked_words(self.description)
        super().save(*args, **kwargs)

    def is_published(self):
        return self.state == "PU"


In [ ]:
python manage.py makemigrations
python manage.py migrate

## Modelo Abstracto como base

In [ ]:
python manage.py startapp base

## base/models.py

In [ ]:
from django.db import models
from django.utils import timezone

class BasePublishModel(models.Model):
    class PublishStateOptions(models.TextChoices):
        PUBLISHED = "PU", "PUBLICADO"
        DRAFT = "BR", "BORRADOR"
        PRIVATE = "PR", "PRIVADO"

    state = models.CharField(
        max_length=2,
        choices=PublishStateOptions.choices,
        default=PublishStateOptions.DRAFT
    )
    timestamp = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)
    publish_timestamp = models.DateTimeField(auto_now=False, null=True)

    class Meta:
        abstract = True
        ordering = ["-updated", "-timestamp"]

    def save(self, *args, **kwargs):
        if self.state_is_published and self.publish_timestamp is None:
            self.publish_timestamp = timezone.now()
        elif not self.state_is_published:
            self.publish_timestamp = None
        super().save(*args, **kwargs)

    @property
    def state_is_published(self):
        return self.state == self.PublishStateOptions.PUBLISHED

    def is_published(self):
        publish_timestamp = self.publish_timestamp
        return self.state_is_published and publish_timestamp < timezone.now()


### ecommerce/models.py

In [ ]:
from django.db import models
from .validators import validate_blocked_words
from base.models import BasePublishModel

class ProductModel(BasePublishModel):
    
    title = models.CharField(max_length=255)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    description = models.TextField(null=True, blank=True)
    seller = models.CharField(max_length=255, null=True, blank=True)
    color = models.CharField(max_length=50, null=True, blank=True)
    product_dimensions = models.CharField(max_length=100, null=True, blank=True)

    def __str__(self):
        return self.title

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        validate_blocked_words(self.description)
        super().save(*args, **kwargs)


### config/settings.py

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig",
    "base.apps.BaseConfig",
    "debug_toolbar",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Creacion a Granel/Bulk creation

In [ ]:
python manage.py shell

In [ ]:
products_data = []
for i in range(1, 100):
    new_data = {"title": "Producto {}".format(i), "price": i*100+99.99}
    products_data.append(new_data)

In [ ]:
from ecommerce.models import ProductModel
new_objects = []
for product_data in products_data:
    print(product_data)
    new_objects.append(ProductModel(**product_data))

In [ ]:
ProductModel.objects.bulk_create(new_objects, ignore_conflicts = True)

## Slugfield y Senales en Modelos

In [ ]:
slug: se usa para los urls
title: escritorio con altura ajustable
slug: /escritorio-con-altura-ajustable
url: www.mywebsite.com/escritorio-con-altura-ajustable
url:www.mywebsite.com/1

Signals / Senales
- pre_save
- post_save
- pre_delete
- post_delete
- pre_init
- post_init
- pre_migrate
- post_migrate

### Ver signals disponibles

In [ ]:
python manage.py shell
from django.db.models import signals
dir(signals)

In [ ]:
['ModelSignal', 
 'Signal',
 '__builtins__', 
 '__cached__', 
 '__doc__', 
 '__file__', 
 '__loader__', 
 '__name__', 
 '__package__', 
 '__spec__', 
 'class_prepared', 
 'm2m_changed', 
 'make_model_tuple', 
 'partial', 
 'post_delete', 
 'post_init', 
 'post_migrate', 
 'post_save', 
 'pre_delete', 
 'pre_init', 
 'pre_migrate',
 'pre_save']

### ecommerce/models.py

In [ ]:
from django.db import models
from django.db.models.signals import pre_save
from django.utils.text import slugify

from base.models import BasePublishModel
from .validators import validate_blocked_words

class ProductModel(BasePublishModel):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)
    slug = models.SlugField(null=True, blank=True, db_index=True)
    seller = models.CharField(max_length=255, null=True, blank=True)
    color = models.CharField(max_length=50, null=True, blank=True)
    product_dimensions = models.CharField(max_length=100, null=True, blank=True)


    def __str__(self):
        return self.title

    def get_absolute_url(self):
        return f"/product/{self.slug}"

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)


# Signal for generating unique slugs
def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyClass = instance.__class__
        qs = MyClass.objects.filter(slug__startwith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(slugify_pre_save, sender=ProductModel)

In [ ]:
python manage.py makemigrations
python manage.py migrate

#### Prueba para crear y ver slugs (prompt)

In [ ]:
from ecommerce.models import ProductModel
obj1 = ProductModel.objects.create(title = "Producto test slug", price = 199.99)
obj1.slug
obj1.slug = ""
obj1.slug
obj1.save()
obj1.slug

obj2 = ProductModel.objects.create(title = "Producto test slug", price = 199.99)
obj2.slug
obj3 = ProductModel.objects.create(title = "Producto test slug", price = 199.99)
obj3.slug
obj4 = ProductModel.objects.create(title = "Producto test slug", price = 199.99)
obj4.slug

### Fixtoures para Cargar Data

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json > ecommerce/fixtures/ProductModel.json

In [ ]:
python manage.py loaddata ecommerce/fixtures/ProductModel.json

#### Eliminar lo que esta ligado a nuestra base de datos (prompt)

from ecommerce.models import ProductModel
ProductModel.objects.all()
ProductModel.objects.all().delete()
ProductModel.objects.all()

#### Llenar la base de datos a partir del fixture

python manage.py loaddata ecommerce/fixtures/ProductModel.json

### Llaves foraneas en modelos

### ecommerce/models.py

In [ ]:
from django.conf import settings
from django.db import models
from django.db.models.signals import pre_save
from django.utils.text import slugify

from base.models import BasePublishModel
from .validators import validate_blocked_words

## User ya esta en django
User = settings.AUTH_USER_MODEL


class ProductModel(BasePublishModel):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField(null=True)
    slug = models.SlugField(null=True, blank=True, db_index=True)
    seller = models.CharField(max_length=255, null=True, blank=True)
    color = models.CharField(max_length=50, null=True, blank=True)
    product_dimensions = models.CharField(max_length=100, null=True, blank=True)
    user = models.ForeignKey(User, null = True, on_delete= models.SET_NULL)
    def __str__(self):
        return self.title

    def get_absolute_url(self):
        return f"/product/{self.slug}"

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)


# Signal for generating unique slugs
def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyClass = instance.__class__
        qs = MyClass.objects.filter(slug__startwith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(slugify_pre_save, sender=ProductModel)

In [ ]:
python manage.py makemigrations

In [ ]:
python manage.py migrate

#### Prueba de Llaves foraneas

In [ ]:
python manage.py shell
from django.contrib.auth import get_user_model
from ecommerce.models import ProductModel
User = get_user_model()
my_user = User.objects.first()

ProductModel.objects.create(user=my_user, title="Producto con Usuario", price=599.99)
ProductModel.objects.create(user=my_user, title="Producto con Usuario", price=599.99)
ProductModel.objects.create(user=my_user, title="Producto con Usuario", price=599.99)

my_user.productmodel_set.all()

qs = ProductModel.objects.filter(user = my_user)
qs

qs.count()




